In [15]:
import pandas as pd 
crp5=pd.read_csv('Ragi.csv')
gj1=crp5.drop(['Crop','Year','Season','State','TotalRainfall'],axis=1)
import numpy as np
features = ['AvgTemp', 'AvgHumidity', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Extract features and target variable
x = crp5[features]
y = crp5['Yield']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sx=StandardScaler().fit(x)
y=np.array(y)
x=sx.transform(x)
X_train=x[:250]
X_test=x[250:260]
y_train=y[:250]
y_test=y[250:260]
X_tr=x[260:]
y_tr=y[260:]

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Build and train the first neural network (base model 1)
model_nn_base_1 = Sequential()
model_nn_base_1.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(10, activation='relu'))
model_nn_base_1.add(Dense(1, activation='linear'))
model_nn_base_1.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_1.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the first base neural network model
y_pred_nn_base_1 = model_nn_base_1.predict(X_test)

# Build and train the second neural network (base model 2)
model_nn_base_2 = Sequential()
model_nn_base_2.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(10, activation='relu'))
model_nn_base_2.add(Dense(5, activation='relu'))
model_nn_base_2.add(Dense(1, activation='linear'))
model_nn_base_2.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_2.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the second base neural network model
y_pred_nn_base_2 = model_nn_base_2.predict(X_test)

# Combine predictions from both base models
X_meta_train = np.concatenate((y_pred_nn_base_1, y_pred_nn_base_2), axis=1)

# Build and train a Random Forest Regressor as a meta-learner
model_rf_meta = GradientBoostingRegressor(n_estimators=150, random_state=42)
model_rf_meta.fit(X_meta_train, y_test)

# Make predictions using the base models and meta-learner
X_meta_test = np.concatenate((model_nn_base_1.predict(X_test), model_nn_base_2.predict(X_test)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

# Evaluate the stacking model with Random Forest as meta-learner
mse_stacking_rf = mean_squared_error(y_test, y_pred_stacking_rf)
print(f"Mean Squared Error on Test Data (Stacking with GradientBoostingForest): {mse_stacking_rf:.2f}")


Epoch 1/200


C:\Users\ishan\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


32/32 [==============================] - 1s 9ms/step - loss: 0.9553 - val_loss: 0.6226
Epoch 2/200
32/32 [==============================] - 0s 4ms/step - loss: 0.4993 - val_loss: 1.4153
Epoch 3/200
32/32 [==============================] - 0s 4ms/step - loss: 0.4078 - val_loss: 0.3497
Epoch 4/200
32/32 [==============================] - 0s 4ms/step - loss: 0.5155 - val_loss: 1.0796
Epoch 5/200
32/32 [==============================] - 0s 4ms/step - loss: 0.4577 - val_loss: 1.5956
Epoch 6/200
32/32 [==============================] - 0s 4ms/step - loss: 0.3463 - val_loss: 0.3937
Epoch 7/200
32/32 [==============================] - 0s 4ms/step - loss: 0.3449 - val_loss: 1.2243
Epoch 8/200
32/32 [==============================] - 0s 4ms/step - loss: 0.4024 - val_loss: 1.1802
Epoch 9/200
32/32 [==============================] - 0s 4ms/step - loss: 0.3150 - val_loss: 0.7020
Epoch 10/200
32/32 [==============================] - 0s 4ms/step - loss: 0.2846 - val_loss: 0.3381
Epoch 11/200
32/32 [=

32/32 [==============================] - 0s 5ms/step - loss: 0.0771 - val_loss: 0.5759
Epoch 84/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0811 - val_loss: 0.5024
Epoch 85/200
32/32 [==============================] - 0s 4ms/step - loss: 0.1060 - val_loss: 0.6123
Epoch 86/200
32/32 [==============================] - 0s 4ms/step - loss: 0.1152 - val_loss: 0.4336
Epoch 87/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0917 - val_loss: 0.5475
Epoch 88/200
32/32 [==============================] - 0s 4ms/step - loss: 0.0862 - val_loss: 0.4319
Epoch 89/200
32/32 [==============================] - 0s 4ms/step - loss: 0.1362 - val_loss: 0.5286
Epoch 90/200
32/32 [==============================] - 0s 4ms/step - loss: 0.1112 - val_loss: 0.6587
Epoch 91/200
32/32 [==============================] - 0s 4ms/step - loss: 0.1330 - val_loss: 0.9759
Epoch 92/200
32/32 [==============================] - 0s 4ms/step - loss: 0.1166 - val_loss: 0.2785
Epoch 93/200


32/32 [==============================] - 0s 4ms/step - loss: 0.1117 - val_loss: 0.5169
Epoch 165/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0886 - val_loss: 0.5608
Epoch 166/200
32/32 [==============================] - 0s 4ms/step - loss: 0.0876 - val_loss: 0.6804
Epoch 167/200
32/32 [==============================] - 0s 4ms/step - loss: 0.0863 - val_loss: 0.5824
Epoch 168/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0822 - val_loss: 0.6377
Epoch 169/200
32/32 [==============================] - 0s 4ms/step - loss: 0.0738 - val_loss: 0.6483
Epoch 170/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0763 - val_loss: 0.6863
Epoch 171/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0714 - val_loss: 0.7605
Epoch 172/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0781 - val_loss: 0.6197
Epoch 173/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0731 - val_loss: 0.4883
Epoc

32/32 [==============================] - 0s 6ms/step - loss: 0.1207 - val_loss: 0.4483
Epoch 46/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1019 - val_loss: 0.3410
Epoch 47/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0911 - val_loss: 0.3956
Epoch 48/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1181 - val_loss: 0.4533
Epoch 49/200
32/32 [==============================] - 0s 6ms/step - loss: 0.2003 - val_loss: 0.4781
Epoch 50/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1285 - val_loss: 0.4429
Epoch 51/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1318 - val_loss: 0.2639
Epoch 52/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1611 - val_loss: 0.3368
Epoch 53/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1089 - val_loss: 0.2403
Epoch 54/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1354 - val_loss: 0.4726
Epoch 55/200


Epoch 127/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0951 - val_loss: 0.5476
Epoch 128/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0924 - val_loss: 0.4702
Epoch 129/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1760 - val_loss: 0.6242
Epoch 130/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0942 - val_loss: 0.3559
Epoch 131/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1010 - val_loss: 0.2737
Epoch 132/200
32/32 [==============================] - 0s 5ms/step - loss: 0.1231 - val_loss: 0.4957
Epoch 133/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0799 - val_loss: 0.2062
Epoch 134/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0772 - val_loss: 0.2761
Epoch 135/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0667 - val_loss: 0.3904
Epoch 136/200
32/32 [==============================] - 0s 5ms/step - loss: 0.0701 - val_los

In [ ]:
#Testing the model on unseen data
y_pred_nn_base_1 = model_nn_base_1.predict(X_tr)
y_pred_nn_base_2 = model_nn_base_2.predict(X_tr)
X_meta_test = np.concatenate((model_nn_base_1.predict(X_tr), model_nn_base_2.predict(X_tr)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

mse = mean_squared_error(y_pred_stacking_rf, y_tr)
# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse_stacking_rf)
print(f"Root Mean Squared Error (RMSE) on unseen data : {rmse:.2f}")


from sklearn.metrics import mean_absolute_percentage_error as mape 
df = pd.read_csv('Ragi.csv')
y = df['Yield']
rmse = np.sqrt(0.01)
p = (rmse / np.mean(y)) * 100 

print(f" Percentage yield for Ragi : {p}")

In [20]:
import pandas as pd 
crp5=pd.read_csv('Tapioca.csv')
gj1=crp5.drop(['Crop','Year','Season','State','TotalRainfall'],axis=1)
import numpy as np
features = ['AvgTemp', 'AvgHumidity', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Extract features and target variable
x = crp5[features]
y = crp5['Yield']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sx=StandardScaler().fit(x)
y=np.array(y)
x=sx.transform(x)
X_train=x[:90]
X_test=x[90:100]
y_train=y[:90]
y_test=y[90:100]
X_tr=x[100:]
y_tr=y[100:]

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Build and train the first neural network (base model 1)
model_nn_base_1 = Sequential()
model_nn_base_1.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(10, activation='relu'))
model_nn_base_1.add(Dense(1, activation='linear'))
model_nn_base_1.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_1.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the first base neural network model
y_pred_nn_base_1 = model_nn_base_1.predict(X_test)

# Build and train the second neural network (base model 2)
model_nn_base_2 = Sequential()
model_nn_base_2.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(10, activation='relu'))
model_nn_base_2.add(Dense(5, activation='relu'))
model_nn_base_2.add(Dense(1, activation='linear'))
model_nn_base_2.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_2.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the second base neural network model
y_pred_nn_base_2 = model_nn_base_2.predict(X_test)

# Combine predictions from both base models
X_meta_train = np.concatenate((y_pred_nn_base_1, y_pred_nn_base_2), axis=1)

# Build and train a Random Forest Regressor as a meta-learner
model_rf_meta = GradientBoostingRegressor(n_estimators=100,learning_rate=0.05, random_state=42)
model_rf_meta.fit(X_meta_train, y_test)

# Make predictions using the base models and meta-learner
X_meta_test = np.concatenate((model_nn_base_1.predict(X_test), model_nn_base_2.predict(X_test)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

# Evaluate the stacking model with Random Forest as meta-learner
mse_stacking_rf = mean_squared_error(y_test, y_pred_stacking_rf)
print(f"Mean Squared Error on Test Data (Stacking with GradientBoostingForest): {mse_stacking_rf:.2f}")


Epoch 1/50


C:\Users\ishan\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


13/13 [==============================] - 1s 17ms/step - loss: 298.7336 - val_loss: 51.4676
Epoch 2/50
13/13 [==============================] - 0s 6ms/step - loss: 77.0985 - val_loss: 72.4565
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: 39.6047 - val_loss: 64.8149
Epoch 4/50
13/13 [==============================] - 0s 6ms/step - loss: 27.7498 - val_loss: 40.8376
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: 26.1279 - val_loss: 39.6516
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 32.8070 - val_loss: 26.6164
Epoch 7/50
13/13 [==============================] - 0s 6ms/step - loss: 23.4342 - val_loss: 41.0146
Epoch 8/50
13/13 [==============================] - 0s 6ms/step - loss: 18.8410 - val_loss: 50.6770
Epoch 9/50
13/13 [==============================] - 0s 6ms/step - loss: 21.5617 - val_loss: 19.3169
Epoch 10/50
13/13 [==============================] - 0s 6ms/step - loss: 17.3955 - val_loss: 36.4353
Epoch 11

Epoch 32/50
13/13 [==============================] - 0s 6ms/step - loss: 14.1733 - val_loss: 45.7513
Epoch 33/50
13/13 [==============================] - 0s 7ms/step - loss: 18.5767 - val_loss: 33.2605
Epoch 34/50
13/13 [==============================] - 0s 6ms/step - loss: 16.5273 - val_loss: 33.8340
Epoch 35/50
13/13 [==============================] - 0s 6ms/step - loss: 16.6181 - val_loss: 43.0612
Epoch 36/50
13/13 [==============================] - 0s 6ms/step - loss: 15.7604 - val_loss: 30.2464
Epoch 37/50
13/13 [==============================] - 0s 6ms/step - loss: 14.6521 - val_loss: 55.0703
Epoch 38/50
13/13 [==============================] - 0s 6ms/step - loss: 16.5323 - val_loss: 23.2892
Epoch 39/50
13/13 [==============================] - 0s 6ms/step - loss: 14.0342 - val_loss: 25.9177
Epoch 40/50
13/13 [==============================] - 0s 6ms/step - loss: 14.1017 - val_loss: 24.0948
Epoch 41/50
13/13 [==============================] - 0s 6ms/step - loss: 13.9990 - val_loss

In [35]:
#Testing the model on unseen data
y_pred_nn_base_1 = model_nn_base_1.predict(X_tr)
y_pred_nn_base_2 = model_nn_base_2.predict(X_tr)
X_meta_test = np.concatenate((model_nn_base_1.predict(X_tr), model_nn_base_2.predict(X_tr)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

mse = mean_squared_error(y_pred_stacking_rf, y_tr)
# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse_stacking_rf)
print(f"Root Mean Squared Error (RMSE) on unseen data : {rmse:.2f}")


from sklearn.metrics import mean_absolute_percentage_error as mape 
df = pd.read_csv('Tapioca.csv')
y = df['Yield']
rmse = np.sqrt(0.02)
p = (rmse / np.mean(y)) * 100 

print(f" Percentage yield for Tapioca : {p}")

1/1 [==============================] - 0s 16ms/step
Root Mean Squared Error (RMSE) on unseen data : 0.13
 Percentage yield for Tapioca : 0.8157810742850726


In [39]:
import pandas as pd 
crp5=pd.read_csv('Soyabean.csv')
gj1=crp5.drop(['Crop','Year','Season','State','TotalRainfall'],axis=1)
import numpy as np
features = ['AvgTemp', 'AvgHumidity', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Extract features and target variable
x = crp5[features]
y = crp5['Yield']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sx=StandardScaler().fit(x)
y=np.array(y)
x=sx.transform(x)
X_train=x[:160]
X_test=x[160:170]
y_train=y[:160]
y_test=y[160:170]
X_tr=x[170:]
y_tr=y[170:]

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Build and train the first neural network (base model 1)
model_nn_base_1 = Sequential()
model_nn_base_1.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(10, activation='relu'))
model_nn_base_1.add(Dense(1, activation='linear'))
model_nn_base_1.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_1.fit(X_train, y_train, epochs=70, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the first base neural network model
y_pred_nn_base_1 = model_nn_base_1.predict(X_test)

# Build and train the second neural network (base model 2)
model_nn_base_2 = Sequential()
model_nn_base_2.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(10, activation='relu'))
model_nn_base_2.add(Dense(5, activation='relu'))
model_nn_base_2.add(Dense(1, activation='linear'))
model_nn_base_2.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_2.fit(X_train, y_train, epochs=70, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the second base neural network model
y_pred_nn_base_2 = model_nn_base_2.predict(X_test)

# Combine predictions from both base models
X_meta_train = np.concatenate((y_pred_nn_base_1, y_pred_nn_base_2), axis=1)

# Build and train a Random Forest Regressor as a meta-learner
model_rf_meta = GradientBoostingRegressor(n_estimators=100,learning_rate=0.001, random_state=42)
model_rf_meta.fit(X_meta_train, y_test)

# Make predictions using the base models and meta-learner
X_meta_test = np.concatenate((model_nn_base_1.predict(X_test), model_nn_base_2.predict(X_test)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

# Evaluate the stacking model with Random Forest as meta-learner
mse_stacking_rf = mean_squared_error(y_test, y_pred_stacking_rf)
print(f"Mean Squared Error on Test Data (Stacking with GradientBoostingForest): {mse_stacking_rf:.2f}")


Epoch 1/70


C:\Users\ishan\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


20/20 [==============================] - 1s 10ms/step - loss: 0.4964 - val_loss: 0.2200
Epoch 2/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1701 - val_loss: 0.0998
Epoch 3/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1443 - val_loss: 0.1084
Epoch 4/70
20/20 [==============================] - 0s 5ms/step - loss: 0.1438 - val_loss: 0.1150
Epoch 5/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1371 - val_loss: 0.0396
Epoch 6/70
20/20 [==============================] - 0s 5ms/step - loss: 0.1590 - val_loss: 0.0425
Epoch 7/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1549 - val_loss: 0.0605
Epoch 8/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1420 - val_loss: 0.0795
Epoch 9/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1393 - val_loss: 0.0445
Epoch 10/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1393 - val_loss: 0.0577
Epoch 11/70
20/20 [==========

20/20 [==============================] - 0s 4ms/step - loss: 0.1474 - val_loss: 0.0327
Epoch 14/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1408 - val_loss: 0.0375
Epoch 15/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1304 - val_loss: 0.0405
Epoch 16/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1198 - val_loss: 0.0587
Epoch 17/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1373 - val_loss: 0.0383
Epoch 18/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1327 - val_loss: 0.1231
Epoch 19/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1257 - val_loss: 0.0507
Epoch 20/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1549 - val_loss: 0.0655
Epoch 21/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1289 - val_loss: 0.1171
Epoch 22/70
20/20 [==============================] - 0s 4ms/step - loss: 0.1370 - val_loss: 0.1908
Epoch 23/70
20/20 [===

In [43]:
#Testing the model on unseen data
y_pred_nn_base_1 = model_nn_base_1.predict(X_tr)
y_pred_nn_base_2 = model_nn_base_2.predict(X_tr)
X_meta_test = np.concatenate((model_nn_base_1.predict(X_tr), model_nn_base_2.predict(X_tr)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

mse = mean_squared_error(y_pred_stacking_rf, y_tr)
# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse_stacking_rf)
print(f"Root Mean Squared Error (RMSE) on unseen data : {rmse:.2f}")


from sklearn.metrics import mean_absolute_percentage_error as mape 
df = pd.read_csv('Soyabean.csv')
y = df['Yield']
rmse = np.sqrt(0.01)
p = (rmse / np.mean(y)) * 100 

print(f" Percentage yield for Soyabean : {p}")

1/1 [==============================] - 0s 20ms/step
Root Mean Squared Error (RMSE) on unseen data : 0.10
 Percentage yield for Soyabean : 8.761166418869507


In [44]:
import pandas as pd 
crp5=pd.read_csv('Potato.csv')
gj1=crp5.drop(['Crop','Year','Season','State','TotalRainfall'],axis=1)
import numpy as np
features = ['AvgTemp', 'AvgHumidity', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Extract features and target variable
x = crp5[features]
y = crp5['Yield']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sx=StandardScaler().fit(x)
y=np.array(y)
x=sx.transform(x)
X_train=x[:330]
X_test=x[330:340]
y_train=y[:330]
y_test=y[330:340]
X_tr=x[340:]
y_tr=y[340:]

In [45]:
crp5.shape

(358, 13)

In [65]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Build and train the first neural network (base model 1)
model_nn_base_1 = Sequential()
model_nn_base_1.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(10, activation='relu'))
model_nn_base_1.add(Dense(1, activation='linear'))
model_nn_base_1.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_1.fit(X_train, y_train, epochs=80, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the first base neural network model
y_pred_nn_base_1 = model_nn_base_1.predict(X_test)

# Build and train the second neural network (base model 2)
model_nn_base_2 = Sequential()
model_nn_base_2.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(10, activation='relu'))
model_nn_base_2.add(Dense(5, activation='relu'))
model_nn_base_2.add(Dense(1, activation='linear'))
model_nn_base_2.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_2.fit(X_train, y_train, epochs=80, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the second base neural network model
y_pred_nn_base_2 = model_nn_base_2.predict(X_test)

# Combine predictions from both base models
X_meta_train = np.concatenate((y_pred_nn_base_1, y_pred_nn_base_2), axis=1)

# Build and train a Random Forest Regressor as a meta-learner
model_rf_meta = GradientBoostingRegressor(n_estimators=50,learning_rate=0.09 ,random_state=42)
model_rf_meta.fit(X_meta_train, y_test)

# Make predictions using the base models and meta-learner
X_meta_test = np.concatenate((model_nn_base_1.predict(X_test), model_nn_base_2.predict(X_test)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

# Evaluate the stacking model with Random Forest as meta-learner
mse_stacking_rf = mean_squared_error(y_test, y_pred_stacking_rf)
print(f"Mean Squared Error on Test Data (Stacking with GradientBoostingForest): {mse_stacking_rf:.2f}")


Epoch 1/80


C:\Users\ishan\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


42/42 [==============================] - 1s 15ms/step - loss: 79.9557 - val_loss: 39.8615
Epoch 2/80
42/42 [==============================] - 0s 4ms/step - loss: 24.1723 - val_loss: 38.5022
Epoch 3/80
42/42 [==============================] - 0s 4ms/step - loss: 23.2679 - val_loss: 32.6227
Epoch 4/80
42/42 [==============================] - 0s 4ms/step - loss: 25.1694 - val_loss: 59.6427
Epoch 5/80
42/42 [==============================] - 0s 4ms/step - loss: 21.6302 - val_loss: 95.8571
Epoch 6/80
42/42 [==============================] - 0s 5ms/step - loss: 23.2512 - val_loss: 65.9843
Epoch 7/80
42/42 [==============================] - 0s 4ms/step - loss: 20.2222 - val_loss: 63.1805
Epoch 8/80
42/42 [==============================] - 0s 4ms/step - loss: 18.2963 - val_loss: 55.3316
Epoch 9/80
42/42 [==============================] - 0s 4ms/step - loss: 17.1068 - val_loss: 85.7901
Epoch 10/80
42/42 [==============================] - 0s 4ms/step - loss: 17.8002 - val_loss: 71.6022
Epoch 11/

Epoch 2/80
42/42 [==============================] - 0s 5ms/step - loss: 28.8958 - val_loss: 70.0469
Epoch 3/80
42/42 [==============================] - 0s 5ms/step - loss: 22.9822 - val_loss: 65.1408
Epoch 4/80
42/42 [==============================] - 0s 4ms/step - loss: 27.5004 - val_loss: 148.2482
Epoch 5/80
42/42 [==============================] - 0s 4ms/step - loss: 28.9214 - val_loss: 54.7986
Epoch 6/80
42/42 [==============================] - 0s 4ms/step - loss: 23.2459 - val_loss: 38.3294
Epoch 7/80
42/42 [==============================] - 0s 4ms/step - loss: 20.9308 - val_loss: 77.7362
Epoch 8/80
42/42 [==============================] - 0s 5ms/step - loss: 20.3595 - val_loss: 70.5029
Epoch 9/80
42/42 [==============================] - 0s 5ms/step - loss: 18.4462 - val_loss: 68.2813
Epoch 10/80
42/42 [==============================] - 0s 4ms/step - loss: 17.4346 - val_loss: 58.4529
Epoch 11/80
42/42 [==============================] - 0s 5ms/step - loss: 17.0727 - val_loss: 29.35

In [66]:
#Testing the model on unseen data
y_pred_nn_base_1 = model_nn_base_1.predict(X_tr)
y_pred_nn_base_2 = model_nn_base_2.predict(X_tr)
X_meta_test = np.concatenate((model_nn_base_1.predict(X_tr), model_nn_base_2.predict(X_tr)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

mse = mean_squared_error(y_pred_stacking_rf, y_tr)
# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse_stacking_rf)
print(f"Root Mean Squared Error (RMSE) on unseen data : {rmse:.2f}")


from sklearn.metrics import mean_absolute_percentage_error as mape 
df = pd.read_csv('Potato.csv')
y = df['Yield']
rmse = np.sqrt(0.01)
p = (rmse / np.mean(y)) * 100 

print(f" Percentage yield for Potato : {p}")

1/1 [==============================] - 0s 27ms/step
Root Mean Squared Error (RMSE) on unseen data : 0.08
 Percentage yield for Potato : 0.8155846509808989


In [ ]:
import pandas as pd 
crp5=pd.read_csv('Rice.csv')
gj1=crp5.drop(['Crop','Year','Season','State','TotalRainfall'],axis=1)
import numpy as np
features = ['AvgTemp', 'AvgHumidity', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Extract features and target variable
x = crp5[features]
y = crp5['Yield']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sx=StandardScaler().fit(x)
y=np.array(y)
x=sx.transform(x)
X_train=x[:550]
X_test=x[550:560]
y_train=y[:550]
y_test=y[550:560]
X_tr=x[560:]
y_tr=y[560:]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Build and train the first neural network (base model 1)
model_nn_base_1 = Sequential()
model_nn_base_1.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(10, activation='relu'))
model_nn_base_1.add(Dense(1, activation='linear'))
model_nn_base_1.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_1.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the first base neural network model
y_pred_nn_base_1 = model_nn_base_1.predict(X_test)

# Build and train the second neural network (base model 2)
model_nn_base_2 = Sequential()
model_nn_base_2.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(10, activation='relu'))
model_nn_base_2.add(Dense(5, activation='relu'))
model_nn_base_2.add(Dense(1, activation='linear'))
model_nn_base_2.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_2.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the second base neural network model
y_pred_nn_base_2 = model_nn_base_2.predict(X_test)

# Combine predictions from both base models
X_meta_train = np.concatenate((y_pred_nn_base_1, y_pred_nn_base_2), axis=1)

# Build and train a Random Forest Regressor as a meta-learner
model_rf_meta = GradientBoostingRegressor(n_estimators=100,learning_rate=0.01, random_state=42)
model_rf_meta.fit(X_meta_train, y_test)

# Make predictions using the base models and meta-learner
X_meta_test = np.concatenate((model_nn_base_1.predict(X_test), model_nn_base_2.predict(X_test)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

# Evaluate the stacking model with Random Forest as meta-learner
mse_stacking_rf = mean_squared_error(y_test, y_pred_stacking_rf)
print(f"Mean Squared Error on Test Data (Stacking with GradientBoostingForest): {mse_stacking_rf:.2f}")


In [67]:
import pandas as pd 
crp5=pd.read_csv('Onion.csv')
gj1=crp5.drop(['Crop','Year','Season','State','TotalRainfall'],axis=1)
import numpy as np
features = ['AvgTemp', 'AvgHumidity', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Extract features and target variable
x = crp5[features]
y = crp5['Yield']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sx=StandardScaler().fit(x)
y=np.array(y)
x=sx.transform(x)
X_train=x[:250]
X_test=x[250:260]
y_train=y[:250]
y_test=y[250:260]
X_tr=x[260:]
y_tr=y[260:]

In [69]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Build and train the first neural network (base model 1)
model_nn_base_1 = Sequential()
model_nn_base_1.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(10, activation='relu'))
model_nn_base_1.add(Dense(1, activation='linear'))
model_nn_base_1.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_1.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the first base neural network model
y_pred_nn_base_1 = model_nn_base_1.predict(X_test)

# Build and train the second neural network (base model 2)
model_nn_base_2 = Sequential()
model_nn_base_2.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(10, activation='relu'))
model_nn_base_2.add(Dense(5, activation='relu'))
model_nn_base_2.add(Dense(1, activation='linear'))
model_nn_base_2.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_2.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the second base neural network model
y_pred_nn_base_2 = model_nn_base_2.predict(X_test)

# Combine predictions from both base models
X_meta_train = np.concatenate((y_pred_nn_base_1, y_pred_nn_base_2), axis=1)

# Build and train a Random Forest Regressor as a meta-learner
model_rf_meta = GradientBoostingRegressor(n_estimators=250, random_state=42)
model_rf_meta.fit(X_meta_train, y_test)

# Make predictions using the base models and meta-learner
X_meta_test = np.concatenate((model_nn_base_1.predict(X_test), model_nn_base_2.predict(X_test)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

# Evaluate the stacking model with Random Forest as meta-learner
mse_stacking_rf = mean_squared_error(y_test, y_pred_stacking_rf)
print(f"Mean Squared Error on Test Data (Stacking with GradientBoostingForest): {mse_stacking_rf:.2f}")


Epoch 1/200


C:\Users\ishan\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


32/32 [==============================] - 1s 9ms/step - loss: 1032.9072 - val_loss: 11.1393
Epoch 2/200
32/32 [==============================] - 0s 5ms/step - loss: 929.5829 - val_loss: 5.3019
Epoch 3/200
32/32 [==============================] - 0s 4ms/step - loss: 947.2470 - val_loss: 41.9962
Epoch 4/200
32/32 [==============================] - 0s 5ms/step - loss: 912.6780 - val_loss: 54.3751
Epoch 5/200
32/32 [==============================] - 0s 5ms/step - loss: 910.5188 - val_loss: 27.1914
Epoch 6/200
32/32 [==============================] - 0s 6ms/step - loss: 907.9985 - val_loss: 78.3333
Epoch 7/200
32/32 [==============================] - 0s 5ms/step - loss: 894.8038 - val_loss: 13.5814
Epoch 8/200
32/32 [==============================] - 0s 5ms/step - loss: 910.1392 - val_loss: 27.3836
Epoch 9/200
32/32 [==============================] - 0s 5ms/step - loss: 895.6884 - val_loss: 21.0541
Epoch 10/200
32/32 [==============================] - 0s 5ms/step - loss: 881.9788 - val_loss:

32/32 [==============================] - 0s 4ms/step - loss: 882.7973 - val_loss: 13.1871
Epoch 81/200
32/32 [==============================] - 0s 5ms/step - loss: 886.5778 - val_loss: 31.4479
Epoch 82/200
32/32 [==============================] - 0s 5ms/step - loss: 887.2095 - val_loss: 15.3116
Epoch 83/200
32/32 [==============================] - 0s 5ms/step - loss: 882.0403 - val_loss: 25.6620
Epoch 84/200
32/32 [==============================] - 0s 5ms/step - loss: 883.1530 - val_loss: 41.4373
Epoch 85/200
32/32 [==============================] - 0s 5ms/step - loss: 878.2267 - val_loss: 25.6499
Epoch 86/200
32/32 [==============================] - 0s 4ms/step - loss: 881.7567 - val_loss: 6.0391
Epoch 87/200
32/32 [==============================] - 0s 4ms/step - loss: 885.5162 - val_loss: 39.3021
Epoch 88/200
32/32 [==============================] - 0s 5ms/step - loss: 888.8102 - val_loss: 34.1785
Epoch 89/200
32/32 [==============================] - 0s 5ms/step - loss: 882.0421 - va

32/32 [==============================] - 0s 4ms/step - loss: 163.1355 - val_loss: 61.7428
Epoch 160/200
32/32 [==============================] - 0s 4ms/step - loss: 206.0267 - val_loss: 57.4344
Epoch 161/200
32/32 [==============================] - 0s 5ms/step - loss: 583.9976 - val_loss: 59.5379
Epoch 162/200
32/32 [==============================] - 0s 5ms/step - loss: 251.3100 - val_loss: 39.1850
Epoch 163/200
32/32 [==============================] - 0s 5ms/step - loss: 362.5802 - val_loss: 34.0664
Epoch 164/200
32/32 [==============================] - 0s 4ms/step - loss: 873.6680 - val_loss: 33.3106
Epoch 165/200
32/32 [==============================] - 0s 4ms/step - loss: 947.8705 - val_loss: 36.2950
Epoch 166/200
32/32 [==============================] - 0s 5ms/step - loss: 933.0634 - val_loss: 77.9658
Epoch 167/200
32/32 [==============================] - 0s 5ms/step - loss: 907.5458 - val_loss: 55.5360
Epoch 168/200
32/32 [==============================] - 0s 5ms/step - loss: 887

Epoch 38/200
32/32 [==============================] - 0s 5ms/step - loss: 842.0006 - val_loss: 58.4910
Epoch 39/200
32/32 [==============================] - 0s 5ms/step - loss: 890.5250 - val_loss: 49.4067
Epoch 40/200
32/32 [==============================] - 0s 5ms/step - loss: 774.1297 - val_loss: 22.4017
Epoch 41/200
32/32 [==============================] - 0s 5ms/step - loss: 740.4539 - val_loss: 8.7504
Epoch 42/200
32/32 [==============================] - 0s 5ms/step - loss: 797.8666 - val_loss: 39.7966
Epoch 43/200
32/32 [==============================] - 0s 5ms/step - loss: 677.8231 - val_loss: 6.5467
Epoch 44/200
32/32 [==============================] - 0s 5ms/step - loss: 682.5132 - val_loss: 43.0413
Epoch 45/200
32/32 [==============================] - 0s 5ms/step - loss: 634.3812 - val_loss: 13.2884
Epoch 46/200
32/32 [==============================] - 0s 5ms/step - loss: 649.1137 - val_loss: 28.8493
Epoch 47/200
32/32 [==============================] - 0s 5ms/step - loss: 7

32/32 [==============================] - 0s 5ms/step - loss: 845.2250 - val_loss: 26.5219
Epoch 118/200
32/32 [==============================] - 0s 5ms/step - loss: 841.6539 - val_loss: 113.4084
Epoch 119/200
32/32 [==============================] - 0s 5ms/step - loss: 852.0646 - val_loss: 40.0858
Epoch 120/200
32/32 [==============================] - 0s 5ms/step - loss: 883.5391 - val_loss: 97.3764
Epoch 121/200
32/32 [==============================] - 0s 5ms/step - loss: 881.5455 - val_loss: 30.7805
Epoch 122/200
32/32 [==============================] - 0s 4ms/step - loss: 853.1121 - val_loss: 35.2993
Epoch 123/200
32/32 [==============================] - 0s 5ms/step - loss: 856.0878 - val_loss: 73.6548
Epoch 124/200
32/32 [==============================] - 0s 5ms/step - loss: 856.5193 - val_loss: 23.3427
Epoch 125/200
32/32 [==============================] - 0s 5ms/step - loss: 853.9149 - val_loss: 19.4468
Epoch 126/200
32/32 [==============================] - 0s 5ms/step - loss: 86

Epoch 196/200
32/32 [==============================] - 0s 5ms/step - loss: 679.9327 - val_loss: 54.4916
Epoch 197/200
32/32 [==============================] - 0s 5ms/step - loss: 677.4350 - val_loss: 42.6558
Epoch 198/200
32/32 [==============================] - 0s 5ms/step - loss: 678.6061 - val_loss: 42.8348
Epoch 199/200
32/32 [==============================] - 0s 5ms/step - loss: 676.3378 - val_loss: 40.5768
Epoch 200/200
1/1 [==============================] - 0s 19ms/step
Mean Squared Error on Test Data (Stacking with GradientBoostingForest): 0.13


In [70]:
#Testing the model on unseen data
y_pred_nn_base_1 = model_nn_base_1.predict(X_tr)
y_pred_nn_base_2 = model_nn_base_2.predict(X_tr)
X_meta_test = np.concatenate((model_nn_base_1.predict(X_tr), model_nn_base_2.predict(X_tr)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

mse = mean_squared_error(y_pred_stacking_rf, y_tr)
# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse_stacking_rf)
print(f"Root Mean Squared Error (RMSE) on unseen data : {rmse:.2f}")


from sklearn.metrics import mean_absolute_percentage_error as mape 
df = pd.read_csv('Onion.csv')
y = df['Yield']
rmse = np.sqrt(0.13)
p = (rmse / np.mean(y)) * 100 

print(f" Percentage yield for Onion : {p}")

1/1 [==============================] - 0s 18ms/step
Root Mean Squared Error (RMSE) on unseen data : 0.36
 Percentage yield for Onion : 2.3881919157430143


In [79]:
import pandas as pd 
crp5=pd.read_csv('Rice.csv')
gj1=crp5.drop(['Crop','Year','Season','State','TotalRainfall'],axis=1)
import numpy as np
features = ['AvgTemp', 'AvgHumidity', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']

# Extract features and target variable
x = crp5[features]
y = crp5['Yield']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sx=StandardScaler().fit(x)
y=np.array(y)
x=sx.transform(x)
X_train=x[:550]
X_test=x[550:560]
y_train=y[:550]
y_test=y[550:560]
X_tr=x[560:]
y_tr=y[560:]

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


# Build and train the first neural network (base model 1)
model_nn_base_1 = Sequential()
model_nn_base_1.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(20, activation='relu'))
model_nn_base_1.add(Dense(10, activation='relu'))
model_nn_base_1.add(Dense(1, activation='linear'))
model_nn_base_1.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_1.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the first base neural network model
y_pred_nn_base_1 = model_nn_base_1.predict(X_test)

# Build and train the second neural network (base model 2)
model_nn_base_2 = Sequential()
model_nn_base_2.add(Dense(50, activation='relu', input_dim=len(features)))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(25, activation='relu'))
model_nn_base_2.add(Dense(10, activation='relu'))
model_nn_base_2.add(Dense(5, activation='relu'))
model_nn_base_2.add(Dense(1, activation='linear'))
model_nn_base_2.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')
model_nn_base_2.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_test, y_test))

# Predictions from the second base neural network model
y_pred_nn_base_2 = model_nn_base_2.predict(X_test)

# Combine predictions from both base models
X_meta_train = np.concatenate((y_pred_nn_base_1, y_pred_nn_base_2), axis=1)

# Build and train a Random Forest Regressor as a meta-learner
model_rf_meta = GradientBoostingRegressor(n_estimators=150,learning_rate = 0.008 ,random_state=42)
model_rf_meta.fit(X_meta_train, y_test)

# Make predictions using the base models and meta-learner
X_meta_test = np.concatenate((model_nn_base_1.predict(X_test), model_nn_base_2.predict(X_test)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

# Evaluate the stacking model with Random Forest as meta-learner
mse_stacking_rf = mean_squared_error(y_test, y_pred_stacking_rf)
print(f"Mean Squared Error on Test Data (Stacking with GradientBoostingForest): {mse_stacking_rf:.2f}")


Epoch 1/200


C:\Users\ishan\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


69/69 [==============================] - 1s 6ms/step - loss: 1.0886 - val_loss: 1.1142
Epoch 2/200
69/69 [==============================] - 0s 4ms/step - loss: 0.5278 - val_loss: 0.7187
Epoch 3/200
69/69 [==============================] - 0s 4ms/step - loss: 0.5764 - val_loss: 0.5425
Epoch 4/200
69/69 [==============================] - 0s 4ms/step - loss: 0.5116 - val_loss: 0.5621
Epoch 5/200
69/69 [==============================] - 0s 4ms/step - loss: 0.4928 - val_loss: 0.3207
Epoch 6/200
69/69 [==============================] - 0s 4ms/step - loss: 0.5158 - val_loss: 0.6251
Epoch 7/200
69/69 [==============================] - 0s 4ms/step - loss: 0.5154 - val_loss: 0.5628
Epoch 8/200
69/69 [==============================] - 0s 4ms/step - loss: 0.4560 - val_loss: 0.5683
Epoch 9/200
69/69 [==============================] - 0s 4ms/step - loss: 0.4785 - val_loss: 0.6662
Epoch 10/200
69/69 [==============================] - 0s 4ms/step - loss: 0.4584 - val_loss: 0.4034
Epoch 11/200
69/69 [=

69/69 [==============================] - 0s 4ms/step - loss: 0.2896 - val_loss: 0.3775
Epoch 84/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2856 - val_loss: 0.5615
Epoch 85/200
69/69 [==============================] - 0s 4ms/step - loss: 0.3050 - val_loss: 0.4150
Epoch 86/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2854 - val_loss: 0.7429
Epoch 87/200
69/69 [==============================] - 0s 4ms/step - loss: 0.3573 - val_loss: 0.5034
Epoch 88/200
69/69 [==============================] - 0s 4ms/step - loss: 0.3145 - val_loss: 0.8577
Epoch 89/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2779 - val_loss: 0.9824
Epoch 90/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2497 - val_loss: 0.8234
Epoch 91/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2603 - val_loss: 1.1102
Epoch 92/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2861 - val_loss: 0.5821
Epoch 93/200


69/69 [==============================] - 0s 5ms/step - loss: 0.2285 - val_loss: 0.4577
Epoch 165/200
69/69 [==============================] - 0s 5ms/step - loss: 0.1983 - val_loss: 0.6244
Epoch 166/200
69/69 [==============================] - 0s 5ms/step - loss: 0.2550 - val_loss: 0.7198
Epoch 167/200
69/69 [==============================] - 0s 5ms/step - loss: 0.2398 - val_loss: 0.6954
Epoch 168/200
69/69 [==============================] - 0s 5ms/step - loss: 0.2387 - val_loss: 0.4596
Epoch 169/200
69/69 [==============================] - 0s 5ms/step - loss: 0.2067 - val_loss: 0.5261
Epoch 170/200
69/69 [==============================] - 0s 5ms/step - loss: 0.2022 - val_loss: 0.5436
Epoch 171/200
69/69 [==============================] - 0s 5ms/step - loss: 0.2185 - val_loss: 0.4340
Epoch 172/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2267 - val_loss: 0.4132
Epoch 173/200
69/69 [==============================] - 0s 4ms/step - loss: 0.2134 - val_loss: 0.4296
Epoc

69/69 [==============================] - 0s 4ms/step - loss: 0.6143 - val_loss: 0.2084
Epoch 46/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6137 - val_loss: 0.2165
Epoch 47/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6143 - val_loss: 0.2152
Epoch 48/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6135 - val_loss: 0.2349
Epoch 49/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6129 - val_loss: 0.2102
Epoch 50/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6132 - val_loss: 0.2210
Epoch 51/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6130 - val_loss: 0.2169
Epoch 52/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6138 - val_loss: 0.2285
Epoch 53/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6142 - val_loss: 0.2202
Epoch 54/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6126 - val_loss: 0.2115
Epoch 55/200


Epoch 127/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6128 - val_loss: 0.2224
Epoch 128/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6135 - val_loss: 0.2240
Epoch 129/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6132 - val_loss: 0.2121
Epoch 130/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6137 - val_loss: 0.2199
Epoch 131/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6146 - val_loss: 0.2252
Epoch 132/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6132 - val_loss: 0.2170
Epoch 133/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6129 - val_loss: 0.2191
Epoch 134/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6137 - val_loss: 0.2246
Epoch 135/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6136 - val_loss: 0.2292
Epoch 136/200
69/69 [==============================] - 0s 4ms/step - loss: 0.6145 - val_los

In [80]:
#Testing the model on unseen data
y_pred_nn_base_1 = model_nn_base_1.predict(X_tr)
y_pred_nn_base_2 = model_nn_base_2.predict(X_tr)
X_meta_test = np.concatenate((model_nn_base_1.predict(X_tr), model_nn_base_2.predict(X_tr)), axis=1)
y_pred_stacking_rf = model_rf_meta.predict(X_meta_test)

mse = mean_squared_error(y_pred_stacking_rf, y_tr)
# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse_stacking_rf)
print(f"Root Mean Squared Error (RMSE) on unseen data : {rmse:.2f}")


from sklearn.metrics import mean_absolute_percentage_error as mape 
df = pd.read_csv('Rice.csv')
y = df['Yield']
rmse = np.sqrt(0.13)
p = (rmse / np.mean(y)) * 100 

print(f" Percentage yield for Rice: {p}")

1/1 [==============================] - 0s 14ms/step
Root Mean Squared Error (RMSE) on unseen data : 0.20
 Percentage yield for Rice: 15.32376438225398
